In [3]:
import csv

with open('train-balanced-sarcasm.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

In [4]:
import string
from nltk.stem.porter import *
from nltk.corpus import stopwords


X = []
Y = []

clean_data = []
punct = string.punctuation
stemmer = PorterStemmer() 
for d in data[1:]: 
    label = d[0]
    text = d[1]
    text = text.lower()
    
    #Remove punctuation
    text = [c for c in text if not (c in punct)]    
    text = ' '.join(text)    
    
    #Stemming and removing stop words gave worse accuracy
    
    #Remove stop words
    #stops = set(stopwords.words("english"))
    #text = [w for w in text if not w in stops and len(w) >= 3]
 
    #Stem the words
    #text = [stemmer.stem(w) for w in text]
    X.append(text)
    Y.append(int(label))

#The model automatically splits train and validation
X_Train = X[:700000]
Y_Train = Y[:700000]
X_Test = X[700000:900000]
Y_Test = Y[700000:900000]

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [6]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_Train)
sequences = tokenizer.texts_to_sequences(X_Train)
dat = pad_sequences(sequences, maxlen=50)

In [ ]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import numpy as np
model.fit(dat, np.array(Y_Train), validation_split=0.4, epochs=3)

Train on 420000 samples, validate on 280000 samples
Epoch 1/3
 77088/420000 [====>.........................] - ETA: 11:08 - loss: 0.6812 - acc: 0.5399

In [ ]:
#Test the model
predict = model.predict(pad_sequences(tokenizer.texts_to_sequences(X_Test),maxlen=50))
correct = 0
output = []
for i in predict:
    if i<0.50:
        output.append(0)
    else:
        output.append(1)
        
print(output[:10])
print(Y_Test[:10])
for i in range(0,len(Y_Test)):
    if output[i] == Y_Test[i]:
        correct+=1
        
print("Test Accuracy: "+ str(correct/len(Y_Test)))